!pip install census

!pip install us

In [28]:
from census import Census
from us import states
import pandas as pd
import numpy as np
import pylab as pl
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#### Insert your census api key her below

In [29]:
c = Census("4c26aa6ebbaef54a55d3903212eabbb506ade381")

#### This is the data dictionary

#### 1990 SF1 Variables

#### Creating the buckets for variables that are separeted into bins like income

In [30]:
buckets = [0, 5000, 10000, 12500, 15000, 17500, 20000, 22500, 25000, 27500, 30000, 32500, 35000, 37500, 40000, 
           42500, 45000, 47500, 50000, 55000, 60000, 75000, 100000, 125000, 150000, 1000000]

#### Insert the variables you want to download. The second part is just so it is automated.

In [31]:
variables = ['H0040001', 'H0080001', 'H0080002', 'H043A001', 'H061A001', 'P0010001',
             'P0060001', 'P0130018', 'P0130019', 'P0130020', 'P0130021', 'P0130022',
             'P0130023', 'P0130024', 'P0130025', 'P0130026', 'P0130027', 'P0130028',
             'P0130029', 'P0130030', 'P0130031', 'P0570006', 'P0800001', 'P0800002',
             'P0800003', 'P0800004', 'P0800005', 'P0800006', 'P0800007', 'P0800008',
             'P0800009', 'P0800010', 'P0800011', 'P0800012', 'P0800013', 'P0800014',
             'P0800015', 'P0800016', 'P0800017', 'P0800018', 'P0800019', 'P0800020',
             'P0800021', 'P0800022', 'P0800023', 'P0800024', 'P0800025', 'P080A001']

In [32]:
var = variables

#### Choose the state and insert its code (as a string number)

In [33]:
state = '09'

In [34]:
sql_query='state:{} county:*'.format(state)

In [35]:
dic = c.sf3.get(var, geo = {'for': 'tract:*',
                       'in': sql_query}, year = 1990)

In [36]:
df = pd.DataFrame.from_dict(dic)

In [37]:
df = df.reset_index()

In [38]:
df = df.drop(columns = 'index')

In [39]:
df['hh_90'] = np.sum(df.iloc[:,22:47],axis=1)
df['pop_over_25'] = np.sum(df.iloc[:,7:20],axis=1)

#### Exporting the raw file

In [40]:
df.to_csv('variables_' + state + '.csv', index = None)

#### Calculating the thresholds for each of the counties

In [41]:
county = df.county.unique()

In [42]:
counties = {}
counties = counties.fromkeys(county)

In [43]:
columns = df.columns

In [44]:
for i in range(0,len(columns)-5):
    df[columns[i]] = df[columns[i]].astype(float)

In [45]:
for i in county:
    counties[i] = np.median(df[df.county == i].P080A001)

In [46]:
df_threshold = pd.DataFrame.from_dict(counties, orient='index')

In [47]:
df_threshold = df_threshold.rename(columns={0:'median'})

In [48]:
#df_typo['vvli_threshold']=0.3*df_typo['median']
df_threshold['vli_threshold'] = 0.5*df_threshold['median']
df_threshold['li_threshold'] = 0.8*df_threshold['median']
df_threshold['mi_threshold'] = 1.2*df_threshold['median']
df_threshold['mhi_threshold'] = 1.5*df_threshold['median']
df_threshold['hi_threshold'] = 2*df_threshold['median']
df_threshold['vhi_threshold'] = 1000000

#### Creating the different typologies based on each county's median income

In [49]:
table = np.zeros((len(county),len(buckets)-1))
for j in range(0, len(counties)):
    col = 1
    for i in range(0, len(buckets)-1):
        if buckets[i+1] < df_threshold.iloc[j, col]:
            table[j, i] = True
        else:
            table[j, i] = (df_threshold.iloc[j, col] - buckets[i]) / (buckets[i+1] - buckets[i])
            col = col+1

In [50]:
values=[]
for i in county:
    with open('variables_'+state+'.csv', 'r') as fi:
        reader = csv.DictReader(fi)
        def creator(reader, c):
            bt = filter(lambda x: x['county'] == c, reader)
            return (list(bt))
        value = creator(reader, i)
    values.append(value)

In [51]:
income_typo = []
for j in range(0, len(county)):
    for i in range(0, len(values[j])):
        typo = []
        summ = 0
        for col in range(0,len(buckets)-1):
            if ((table[j, col] == 1) & (values[j][i]['hh_90']!='0.0')):
                summ = summ + float(values[j][i][var[22+col]])/float(values[j][i]['hh_90'])
            else:
                if values[j][i]['hh_90']!='0.0':
                    summ = summ + float(values[j][i][var[22+col]])*table[j, col]/float(values[j][i]['hh_90'])
                    typo.append(summ)
                    summ = float(values[j][i][var[22+col]])*(1-table[j, col])/float(values[j][i]['hh_90'])
        typo.append(summ)
        income_typo.append(typo)

In [52]:
df_final = pd.DataFrame(income_typo)

In [53]:
df_final = df_final.rename(columns={0:'vli_90', 1:'li_90', 2:'mi_90', 3:'mhi_90', 4:'hi_90', 5:'vhi_90'})

In [54]:
moe_names = ['per_col_90', 'per_rent_90', 'per_owners_90', 'per_nonwhite_90']
moe_dividend = ['P0570006', 'H0080002', 'H0080001', 'P0060001']
moe_divisor = ['pop_over_25', 'H0040001', 'H0040001', 'P0010001']

In [55]:
for i in range(0,len(moe_names)):
    if i!=3:
        df_final[moe_names[i]] = df[moe_dividend[i]]/df[moe_divisor[i]]
    else:
        df_final[moe_names[i]] = 1 - df[moe_dividend[i]]/df[moe_divisor[i]]

In [56]:
df = df.rename(columns={'H043A001':'mrent_90', 'P080A001':'hinc_90', 
                        'H061A001':'mhval_90', 'H0040001':'hu_90', 'P0010001':'pop_90'})

In [57]:
df_final = df_final.join(df[['hh_90', 'mrent_90', 'hinc_90', 'mhval_90', 'hu_90', 'pop_90', 'state', 'county',
                             'tract']])

#### Droping null values

In [58]:
prev_len = len(df_final)

In [59]:
df_final = df_final.dropna(axis = 0, how = 'any')

In [60]:
final_len = len(df_final)

In [61]:
print('The percentage of information lost is: {}'.format((prev_len-final_len)*100/prev_len))

The percentage of information lost is: 1.3189448441247003


#### Exporting the final file

In [62]:
df_final.to_csv(state+'_1990_variables.csv', index = None)